In [ ]:
!pip install catboost

     |████████████████████████████████| 66.1MB 49kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import necessary dependecies

from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
import os
import warnings
import numpy as np  
import seaborn as sns
import pandas as pd, os, gc
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline
warnings.filterwarnings('ignore')
import tensorflow as tf
import random
from tqdm import tqdm 
import copy
from google.colab import files
 
tf.random.set_seed(111)
np.random.seed(111)
random.seed(111)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#Data 
train_path='Train.csv'
test_path='Test.csv'
submission_path='SampleSubmission.csv'

In [ ]:
def read_train():
  train =pd.read_csv(train_path, parse_dates=['join_date'])
  return train
 
def read_test():
  test =pd.read_csv(test_path, parse_dates=['join_date'])
  return test
 
def read_submission():
  submission =pd.read_csv(submission_path)
  return submission
 
train = read_train();test=read_test();submission=read_submission()

In [ ]:
print(train.shape)

(29132, 29)


In [ ]:
X_train = []
X_train_columns = train.columns
c = 0
for v in train.values:
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for i in index:
    c+=1
    for k in range(len(binary)):
      if k == i:
        binary_transformed = list(copy.copy(binary))
        binary_transformed[i] = 0
        X_train.append(list(info) + binary_transformed + [X_train_columns[8+k]] + [c])
 
X_train = pd.DataFrame(X_train)
X_train.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'ID2']

In [ ]:
X_test = []
true_values = []
c = 0
for v in test.values:
  c += 1
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  X_test.append(list(info) + list(binary) + [c])
  for k in test.columns[8:][index]:
    true_values.append(v[0] + ' X ' + k)
 
X_test = pd.DataFrame(X_test)
X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'ID2']

In [ ]:
len(true_values)

12853

In [ ]:
features_train = []
features_test = []
columns = []
 
append_features = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 
'N2MW', 'AHXO','BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 
'ECY3', 'ID', 'ID2', 'join_date', 'sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code',
'birth_year']
 
for v in append_features:
  features_train.append(X_train[v].values.reshape(-1, 1))
  features_test.append(X_test[v].values.reshape(-1, 1))
  columns.append(np.array([v]))
 
y_train = X_train[['product_pred']]

In [ ]:
features_train = np.concatenate(features_train, axis=1)
features_test = np.concatenate(features_test, axis=1)
columns = np.concatenate(np.array(columns))
 
X_train = pd.DataFrame(features_train)
X_train.columns = columns
X_test = pd.DataFrame(features_test)
X_test.columns = columns

In [ ]:
print(X_test.shape)
X_test.head()

(10000, 30)


,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,join_date,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,F86J5PC,1,1515715200000000000,M,M,94KC,DZRV,90QI,1984
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,H6141K3,2,1547078400000000000,M,M,1X1H,J9SY,90QI,1996
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,RBAYUXZ,3,1577836800000000000,F,W,UAOD,2A7I,T4MS,1968
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,KCBILBQ,4,1546387200000000000,M,M,94KC,2A7I,T4MS,1989
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,LSEC1ZJ,5,1577923200000000000,F,M,UAOD,0KID,T4MS,1982


In [ ]:
X_train.join_date = pd.to_datetime(X_train.join_date, format="%Y-%m-%d")
X_test.join_date = pd.to_datetime(X_test.join_date, format="%Y-%m-%d")

In [ ]:
X_train.tail(3)

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,join_date,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year
66350,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,BUQ8DQN,66351,2019-01-07,M,M,UAOD,2A7I,T4MS,1989
66351,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,DEJBWYU,66352,2020-01-03,M,M,94KC,31JW,90QI,1975
66352,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,DEJBWYU,66353,2020-01-03,M,M,94KC,31JW,90QI,1975


In [ ]:
#Extracting new features
 
X_train['day_of_week'] = X_train['join_date'].dt.dayofweek
X_train['day_of_week_name'] = X_train['join_date'].dt.day_name()
X_train['age'] = np.abs(X_train['join_date'].dt.year - X_train['birth_year'])
 
 
X_test['day_of_week'] = X_test['join_date'].dt.dayofweek
X_test['day_of_week_name'] = X_test['join_date'].dt.day_name()
X_test['age'] = np.abs( X_test['join_date'].dt.year - X_test['birth_year'])

In [ ]:
X_test.head()

,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,ID,ID2,join_date,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,day_of_week,day_of_week_name,age
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,F86J5PC,1,2018-01-12,M,M,94KC,DZRV,90QI,1984,4.0,Friday,34
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,H6141K3,2,2019-01-10,M,M,1X1H,J9SY,90QI,1996,3.0,Thursday,23
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,RBAYUXZ,3,2020-01-01,F,W,UAOD,2A7I,T4MS,1968,2.0,Wednesday,52
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,KCBILBQ,4,2019-01-02,M,M,94KC,2A7I,T4MS,1989,2.0,Wednesday,30
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,LSEC1ZJ,5,2020-01-02,F,M,UAOD,0KID,T4MS,1982,3.0,Thursday,38


In [ ]:
# LABEL ENCODE
enc = LabelEncoder()
def encode_LE(train,test,cols,verbose=True):
  for col in cols:
 
    df_comb = pd.concat([train[col].astype('str'),test[col].astype('str')],axis=0)
    df_comb = enc.fit_transform(df_comb)
    nm = col
    if df_comb.max()>32000: 
      train[nm] = df_comb[:len(train)].astype('int32')
      test[nm] = df_comb[len(train):].astype('int32')
    else:
      train[nm] = df_comb[:len(train)].astype('int16')
      test[nm] = df_comb[len(train):].astype('int16')
    del df_comb; x=gc.collect()
    if verbose: print(nm,', ',end='')

In [ ]:
X_train.day_of_week_name = X_train.day_of_week_name.astype('str')
X_test.day_of_week_name = X_test.day_of_week_name.astype('str')
X_train.join_date = X_train.join_date.astype('str')
X_test.join_date = X_test.join_date.astype('str')

In [ ]:
data = X_train.append(X_test)
for v in ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code','day_of_week_name','join_date']:
  data.loc[:,v] = enc.fit_transform(data.loc[:,v])
X_train = data[:X_train.shape[0]]
X_test = data[-X_test.shape[0]:]

In [ ]:
enc.fit(y_train.iloc[:,0])
y_train = pd.DataFrame(enc.transform(y_train.iloc[:,0]))
y_train.columns = ['target']

In [ ]:
X = X_train.drop(['ID','ID2'], axis=1)

In [ ]:
test = X_test.drop(['ID','ID2'], axis=1)

In [ ]:
print(X.shape)
X.head()

(66353, 31)


,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,join_date,sex,marital_status,branch_code,occupation_code,occupation_category_code,birth_year,day_of_week,day_of_week_name,age
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,114,0,2,0,19,5,1987,2.0,6,32
1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,114,0,2,0,19,5,1987,2.0,6,32
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,118,0,2,11,19,5,1981,6.0,3,38
3,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,118,0,2,11,19,5,1981,6.0,3,38
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,44,1,6,3,185,1,1991,6.0,3,22


In [ ]:
df_train =X.values
df_test =test.values

In [ ]:
y = y_train.target

In [ ]:
import lightgbm as lgb 
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold



seed = 111


sk = StratifiedKFold(n_splits= 5,random_state=seed,shuffle=True)


gbm = xgb.XGBClassifier(
objective = 'multi:softmax',
base_score = np.mean(y),
eval_metric ="mlogloss",
subsample= 0.8,
n_estimators = 2000,
seed=seed,
random_state = seed,
num_class = 21,
)

catboost_model = CatBoostClassifier(random_state=seed,depth=6) 


lgb_clf = lgb.LGBMClassifier(
    learning_rate = 0.015,
    n_estimators = 3000,
    objective ='multiclass',
    random_state = 111,
    num_leaves = 127,
    max_depth = 6,
    metric = 'multi_logloss'
)


In [ ]:
# TRAINING LGBM

lgb_predictions = list()


for fold, (train_idx, test_idx) in enumerate(sk.split(df_train,y)):
  lgb_clf.fit(
      df_train[train_idx],
      y[train_idx],
      eval_set = [(df_train[test_idx], y[test_idx])],
      early_stopping_rounds  = 100,
      categorical_feature = [24,25,26]

  )
  preds = lgb_clf.predict_proba(df_test)
  lgb_predictions.append(preds)


preds_lgbm = np.average(lgb_predictions, axis=0)
print(preds_lgbm.shape)

In [ ]:
y_test = pd.DataFrame(preds_lgbm)
y_test.columns = enc.inverse_transform(y_test.columns)
 
answer_mass = []
for i in range(X_test.shape[0]):
  id = X_test['ID'].iloc[i]
  for c in y_test.columns:
    answer_mass.append([id + ' X ' + str(c), y_test[c].iloc[i]])
 
df_answer = pd.DataFrame(answer_mass)
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
  if df_answer['ID X PCODE'].iloc[i] in true_values:
    df_answer['Label'].iloc[i] = 1.0
 

df_answer.reset_index(drop=True, inplace=True)
 
df_answer.to_csv('submission_lgbm.csv',index=False) 
files.download('submission_lgbm.csv')

In [ ]:
# TRAINING GBM
 
gbm_predictions = list()
 
for fold, (train_idx, test_idx) in enumerate(sk.split(df_train,y)):
  gbm.fit(
      df_train[train_idx],
      y[train_idx],
      eval_set = [(df_train[test_idx], y[test_idx])],
      early_stopping_rounds  = 100,
  )
  preds = gbm.predict_proba(df_test)
  gbm_predictions.append(preds)
 
preds_gbm = np.average(gbm_predictions, axis=0)
print(preds_gbm.shape)

In [ ]:
y_test = pd.DataFrame(preds_gbm)
y_test.columns = enc.inverse_transform(y_test.columns)
 
answer_mass = []
for i in range(X_test.shape[0]):
  id = X_test['ID'].iloc[i]
  for c in y_test.columns:
    answer_mass.append([id + ' X ' + str(c), y_test[c].iloc[i]])
 
df_answer = pd.DataFrame(answer_mass)
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
  if df_answer['ID X PCODE'].iloc[i] in true_values:
    df_answer['Label'].iloc[i] = 1.0
 

df_answer.reset_index(drop=True, inplace=True)
 
df_answer.to_csv('submission_gbm.csv',index=False) 
files.download('submission_gbm.csv')

In [ ]:
# TRAINING CATBOOST
 
catboost_model.fit(
    X,
    y,
    early_stopping_rounds = 100,
    cat_features = [24,25,26],
    
)
 
preds_cat = catboost_model.predict_proba(test)
print(preds_cat.shape)

In [ ]:
 y_test = pd.DataFrame(preds_cat)
y_test.columns = enc.inverse_transform(y_test.columns)
 
answer_mass = []
for i in range(X_test.shape[0]):
  id = X_test['ID'].iloc[i]
  for c in y_test.columns:
    answer_mass.append([id + ' X ' + str(c), y_test[c].iloc[i]])
 
df_answer = pd.DataFrame(answer_mass)
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
  if df_answer['ID X PCODE'].iloc[i] in true_values:
    df_answer['Label'].iloc[i] = 1.0
 
 
df_answer.reset_index(drop=True, inplace=True)
 
df_answer.to_csv('submission_catboost.csv',index=False) 
files.download('submission_catboost.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Training Voting Classifier
 
voting_clf = VotingClassifier([('lgbm',lgb_clf),('gbm',gbm),('catboost',catboost_model)] ,voting = 'soft')
 
voting_clf.fit(df_train,y)
 
 
preds_vc = voting_clf.predict_proba(df_test)
 
print(preds_vc.shape)

(10000, 21)


In [ ]:
y_test = pd.DataFrame(preds_vc)
y_test.columns = enc.inverse_transform(y_test.columns)
 
answer_mass = []
for i in range(X_test.shape[0]):
  id = X_test['ID'].iloc[i]
  for c in y_test.columns:
    answer_mass.append([id + ' X ' + str(c), y_test[c].iloc[i]])
 
df_answer = pd.DataFrame(answer_mass)
df_answer.columns = ['ID X PCODE', 'Label']
for i in range(df_answer.shape[0]):
  if df_answer['ID X PCODE'].iloc[i] in true_values:
    df_answer['Label'].iloc[i] = 1.0
 
 
df_answer.reset_index(drop=True, inplace=True)
 
df_answer.to_csv('submission_voting.csv',index=False) 
files.download('submission_voting.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Ensembling the all models
 
sub1 = pd.read_csv('/content/submission_lgbm.csv')
sub2 = pd.read_csv('/content/submission_gbm.csv')
sub3 = pd.read_csv('/content/submission_catboost.csv')
sub4 = pd.read_csv('/content/submission_voting.csv')
 
sub1['Label'] = 0.4*sub1['Label'] + 0.2*sub2['Label'] + 0.3*sub3['Label'] + 0.1*sub4['Label']

In [ ]:
sub1.to_csv('best.csv', index = False)

In [ ]:
sub1.head(3)

,ID X PCODE,Label
0,F86J5PC X 66FJ,0.000066
1,F86J5PC X 7POT,0.000077
2,F86J5PC X 8NN1,0.000004
